In [1]:
import os
import cv2
import numpy as np
from keras.applications.resnet50 import ResNet50
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.utils import to_categorical
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
def load_data(folder, target_size=(224, 224)):
    data = []
    labels = []
    for label in os.listdir(folder):
        label_folder = os.path.join(folder, label)
        if os.path.isdir(label_folder):
            print(f"Loading images from folder: {label_folder}")
            for filename in os.listdir(label_folder):
                if filename.endswith(".jpg"):
                    img = cv2.imread(os.path.join(label_folder, filename))
                    if img is not None:
                        img = cv2.resize(img, target_size)
                        data.append(img)
                        labels.append(label)
                    else:
                        print(f"Unable to load image: {os.path.join(label_folder, filename)}")
    return np.array(data), np.array(labels)

# Load the collected data
data, labels = load_data(r'C:\Users\ganta\Documents\SignLang\Data')

Loading images from folder: C:\Users\ganta\Documents\SignLang\Data\A
Loading images from folder: C:\Users\ganta\Documents\SignLang\Data\B
Loading images from folder: C:\Users\ganta\Documents\SignLang\Data\C
Loading images from folder: C:\Users\ganta\Documents\SignLang\Data\D
Loading images from folder: C:\Users\ganta\Documents\SignLang\Data\E
Loading images from folder: C:\Users\ganta\Documents\SignLang\Data\F
Loading images from folder: C:\Users\ganta\Documents\SignLang\Data\G
Loading images from folder: C:\Users\ganta\Documents\SignLang\Data\H
Loading images from folder: C:\Users\ganta\Documents\SignLang\Data\I
Loading images from folder: C:\Users\ganta\Documents\SignLang\Data\J
Loading images from folder: C:\Users\ganta\Documents\SignLang\Data\K
Loading images from folder: C:\Users\ganta\Documents\SignLang\Data\L
Loading images from folder: C:\Users\ganta\Documents\SignLang\Data\M
Loading images from folder: C:\Users\ganta\Documents\SignLang\Data\N
Loading images from folder: C:\Use

In [3]:
print("Number of images loaded:", len(data))

# Encode labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

print("Labels:", label_encoder.classes_)

Number of images loaded: 10400
Labels: ['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R'
 'S' 'T' 'U' 'V' 'W' 'X' 'Y' 'Z']


In [4]:
X_train, X_test, y_train, y_test = train_test_split(data, labels_encoded, test_size=0.2, random_state=42) 
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

print("Training set size:", len(X_train))
print("Testing set size:", len(X_test)) 
print("Validation set size:", len(X_val))

Training set size: 6656
Testing set size: 2080
Validation set size: 1664


In [5]:
# Build CNN model
model = models.Sequential([ layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)), 
                           layers.MaxPooling2D((2, 2)), layers.Conv2D(64, (3, 3), activation='relu'),
                           layers.MaxPooling2D((2, 2)), layers.Conv2D(128, (3, 3), activation='relu'), 
                           layers.MaxPooling2D((2, 2)), 
                           layers.Flatten(),
                           layers.Dense(512, activation='relu'),
                           layers.Dense(len(label_encoder.classes_), activation='softmax') ])
# Compile the model
model.compile(optimizer='adam',  loss='categorical_crossentropy', metrics=['accuracy'])

C:\Users\ganta\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [6]:
datagen = ImageDataGenerator(
    rotation_range=15,  
    width_shift_range=0.1,  
    height_shift_range=0.1,  
    shear_range=0.1, 
    zoom_range=0.1,  
    horizontal_flip=True,  
    vertical_flip=True, 
    fill_mode='nearest' 
)

In [17]:
# Train the model
epochs = 10
history_cnn = model.fit(datagen.flow(X_train, tf.keras.utils.to_categorical(y_train, len(label_encoder.classes_)), batch_size=32),
                  epochs=epochs, steps_per_epoch=len(X_train) // 32, validation_data=(X_val, tf.keras.utils.to_categorical(y_val, len(label_encoder.classes_))),
                validation_steps=len(X_test) // 32)

Epoch 1/10
208/208 ━━━━━━━━━━━━━━━━━━━━ 159s 754ms/step - accuracy: 0.6061 - loss: 1.2445 - val_accuracy: 0.8817 - val_loss: 0.2713
Epoch 2/10


C:\Users\ganta\anaconda3\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


208/208 ━━━━━━━━━━━━━━━━━━━━ 8s 41ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.8817 - val_loss: 0.2713
Epoch 3/10
208/208 ━━━━━━━━━━━━━━━━━━━━ 159s 754ms/step - accuracy: 0.8916 - loss: 0.3494 - val_accuracy: 0.9521 - val_loss: 0.0819
Epoch 4/10
208/208 ━━━━━━━━━━━━━━━━━━━━ 8s 37ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.9521 - val_loss: 0.0819
Epoch 5/10
208/208 ━━━━━━━━━━━━━━━━━━━━ 142s 674ms/step - accuracy: 0.9349 - loss: 0.1980 - val_accuracy: 0.9722 - val_loss: 0.0330
Epoch 6/10
208/208 ━━━━━━━━━━━━━━━━━━━━ 7s 35ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.9722 - val_loss: 0.0330
Epoch 7/10
208/208 ━━━━━━━━━━━━━━━━━━━━ 149s 712ms/step - accuracy: 0.9468 - loss: 0.1625 - val_accuracy: 0.9840 - val_loss: 0.0044
Epoch 8/10
208/208 ━━━━━━━━━━━━━━━━━━━━ 8s 41ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.9840 - val_loss: 0.0044
Epoch 9/10
208/208 ━━━━━━━━━━━━━━━━━━━━ 151s 717ms/step - accuracy:

In [18]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, tf.keras.utils.to_categorical(y_test, len(label_encoder.classes_)))
print(f'Test accuracy: {test_accuracy}')

65/65 ━━━━━━━━━━━━━━━━━━━━ 7s 105ms/step - accuracy: 0.9973 - loss: 0.0123
Test accuracy: 0.9966346025466919


In [21]:
# Extract the validation accuracy
validation_accuracy = history_cnn.history['val_accuracy'][-1]
print(f'validation accuracy: {validation_accuracy:.4f}')

validation accuracy: 0.9805


In [7]:
from keras.layers import BatchNormalization, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

In [8]:
num_classes = len(labels)
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)
y_val = to_categorical(y_val, num_classes)

In [9]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the first 100 layers and fine-tune the rest
for layer in base_model.layers[:100]:
    layer.trainable = False
for layer in base_model.layers[100:]:
    layer.trainable = True

# Add custom head to the model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)

# Output layer with softmax activation for classification
predictions = Dense(num_classes, activation='softmax')(x)

# Define the model
model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
# Train the model with data augmentation and early stopping
history_restnet50 = model.fit(datagen.flow(X_train, y_train, batch_size=32),
                              validation_data=(X_val, y_val),
                              epochs=5,
                              callbacks=[EarlyStopping(monitor='val_loss', patience=3)])

Epoch 1/5


C:\Users\ganta\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


208/208 ━━━━━━━━━━━━━━━━━━━━ 559s 3s/step - accuracy: 0.7331 - loss: 3.7952 - val_accuracy: 1.0000 - val_loss: 0.0213
Epoch 2/5
208/208 ━━━━━━━━━━━━━━━━━━━━ 625s 3s/step - accuracy: 0.9996 - loss: 0.0189 - val_accuracy: 1.0000 - val_loss: 9.6630e-04
Epoch 3/5
208/208 ━━━━━━━━━━━━━━━━━━━━ 638s 3s/step - accuracy: 1.0000 - loss: 0.0062 - val_accuracy: 1.0000 - val_loss: 3.3945e-04
Epoch 4/5
208/208 ━━━━━━━━━━━━━━━━━━━━ 618s 3s/step - accuracy: 1.0000 - loss: 0.0033 - val_accuracy: 1.0000 - val_loss: 2.8099e-04
Epoch 5/5
208/208 ━━━━━━━━━━━━━━━━━━━━ 592s 3s/step - accuracy: 0.9983 - loss: 0.0142 - val_accuracy: 1.0000 - val_loss: 3.3656e-04


In [11]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f'Test accuracy: {test_accuracy}')

# Evaluate the model on the validation set
val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=2) 

print(f'validation accuracy: {val_accuracy:.4f}')

65/65 - 65s - 998ms/step - accuracy: 1.0000 - loss: 3.8570e-04
Test accuracy: 1.0
52/52 - 54s - 1s/step - accuracy: 1.0000 - loss: 3.3656e-04
validation accuracy: 1.0000


In [25]:
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, Flatten

In [27]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers for classification
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


history_vgg = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=5, batch_size=32)

Epoch 1/5
208/208 ━━━━━━━━━━━━━━━━━━━━ 984s 5s/step - accuracy: 0.9464 - loss: 0.8515 - val_accuracy: 1.0000 - val_loss: 2.4648e-06
Epoch 2/5
208/208 ━━━━━━━━━━━━━━━━━━━━ 997s 5s/step - accuracy: 1.0000 - loss: 1.1145e-08 - val_accuracy: 1.0000 - val_loss: 1.5591e-06
Epoch 3/5
208/208 ━━━━━━━━━━━━━━━━━━━━ 1004s 5s/step - accuracy: 1.0000 - loss: 1.9186e-11 - val_accuracy: 1.0000 - val_loss: 7.6474e-07
Epoch 4/5
208/208 ━━━━━━━━━━━━━━━━━━━━ 1001s 5s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 7.1313e-07
Epoch 5/5
208/208 ━━━━━━━━━━━━━━━━━━━━ 1071s 5s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 7.0239e-07


In [28]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy: {:.2f}%".format(test_accuracy * 100))

# Evaluate the model on the validation set
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print("Validation Accuracy: {:.2f}%".format(val_accuracy * 100))

65/65 ━━━━━━━━━━━━━━━━━━━━ 199s 3s/step - accuracy: 1.0000 - loss: 4.7465e-06
Test Accuracy: 100.00%
52/52 ━━━━━━━━━━━━━━━━━━━━ 204s 4s/step - accuracy: 1.0000 - loss: 1.0449e-07
Validation Accuracy: 100.00%
